In [ ]:
# Created on Sep 2021
# author: 임일
# modifier : KatieMinjoo

In [1]:
import numpy as np
import pandas as pd

In [3]:
# Load the u.user file into a dataframe
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./u.user', sep = '|', names = u_cols, encoding = 'latin-1')

In [5]:
# Load the u.items file into a dataframe
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('./u.item', sep='|', names=i_cols, encoding='latin-1')

In [6]:
# movie ID와 title을 제외한 컬럼 지우기
movies = movies[['movie_id', 'title']]

In [7]:
# Load the u.data file into a dataframe
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./u.data', sep='\t', names=r_cols, encoding='latin-1')

In [8]:
# Delete timestamp
ratings = ratings.drop('timestamp', axis=1)

In [9]:
# import the train_test_split func
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']

# Split the data into train/test datset (Stratified : used for classification dataset to split every classes in a different set as a equal ratio)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.25, stratify=y)

In [10]:
# Def RMSE
import numpy as np
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [11]:
# Baseline Model (return the mean value of the rating of each movie, if there's no ratings, then it returns 3)
def baseline(user_id, movie_id):
    try:
        rating = train_mean[movie_id]
    except:
        rating = 3.0
    return rating

In [ ]:
# 